In [ ]:
# Install Libraries

# mount Google Drive
from google.colab import drive
drive.mount('/content/colabIntern')
#%cd "/content/colabIntern/MyDrive/2023/Langara/Capstone Project/Collab"
#%cd "/content/colabIntern/MyDrive/Capstone/Capstone Project/Collab"
!ls | wc -l

Drive already mounted at /content/colabIntern; to attempt to forcibly remount, call drive.mount("/content/colabIntern", force_remount=True).
/content/colabIntern/.shortcut-targets-by-id/1lwwFhOdnZo3Bx0mzGW8l1qPio2d8s1TV/Langara/Capstone Project/Collab
6


In [ ]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow       as tf
import keras.optimizers as op
from keras.callbacks import EarlyStopping

from tensorflow                     import keras
from tensorflow.keras.models        import Sequential
from tensorflow.keras.layers        import Dense
from tensorflow.keras.layers        import Dropout
from tensorflow.keras.optimizers    import Adam

In [ ]:
# Load Dataset
df = pd.read_csv('..//..//df.csv')

In [ ]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META', 'PLTR']
tickers.sort()

In [ ]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]
#df=df.iloc[::,:74]
#df.reset_index(inplace=True)

In [ ]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [ ]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks.
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [ ]:
# Select the length of the df ; For this file we need 7 months
# We use the first 3 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 7 months needed)
df = df.sort_index().loc['2022-06-01':'2022-12-30']

In [ ]:
#Set the DATETIME for fecha
DATETIME = df.index.values

In [ ]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day
#(i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [ ]:
df=df.iloc[:,:74]

In [ ]:
num_companies = 29  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(df['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(df['2022-06-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model = Sequential()
model.add(Dense(100, activation = 'relu', input_dim = df.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model.add(Dense(100, activation = 'relu', input_dim = df.iloc[:,1:-1].shape[1]))
model.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = df.iloc[i:train_rows + i, 1:]
    test   = df.iloc[train_rows + i:train_rows + i + step_rows, 1:]
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]



    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model.fit(X_train, y_train, epochs=8, batch_size=256) #, callbacks=[es]
    y_hat = model.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    result = pd.concat([result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

1392/1392 [==============================] - 5s 3ms/step - loss: 5421.8350
Count Down: 81
1392/1392 [==============================] - 4s 3ms/step - loss: 4745.3442
Count Down: 80
1392/1392 [==============================] - 4s 3ms/step - loss: 4692.0112
Count Down: 79
1392/1392 [==============================] - 4s 3ms/step - loss: 4666.5210
Count Down: 78
1392/1392 [==============================] - 4s 3ms/step - loss: 4662.9468
Count Down: 77
1392/1392 [==============================] - 4s 3ms/step - loss: 4649.3271
Count Down: 76
1392/1392 [==============================] - 4s 3ms/step - loss: 4663.1118
Count Down: 75
1392/1392 [==============================] - 4s 3ms/step - loss: 4667.6270
Count Down: 74
1392/1392 [==============================] - 4s 3ms/step - loss: 4674.7959
Count Down: 73
1392/1392 [==============================] - 4s 3ms/step - loss: 4695.5278
Count Down: 72
1392/1392 [==============================] - 5s 3ms/step - loss: 4721.4292
Count Down: 71
1392/1392 

In [ ]:
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [ ]:
result[25::29]

,DATETIME,ACTUAL,PREDICTED,DIFFERENCE,TRAIN_DURATION
25,2022-09-01 09:30:00,274.66,97.231506,177.428494,5.044589
54,2022-09-01 09:45:00,273.62,91.573677,182.046323,5.044589
83,2022-09-01 10:00:00,271.36,97.041275,174.318725,5.044589
112,2022-09-01 10:15:00,273.14,98.788551,174.351449,5.044589
141,2022-09-01 10:30:00,269.54,96.902603,172.637397,5.044589
...,...,...,...,...,...
56256,2022-12-30 14:15:00,121.46,105.495811,15.964189,4.241694
56285,2022-12-30 14:30:00,121.30,105.588379,15.711621,4.241694
56314,2022-12-30 14:45:00,121.51,104.301826,17.208174,4.241694
56343,2022-12-30 15:00:00,122.42,108.782242,13.637758,4.241694


In [ ]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [ ]:
predicted, actual,duration=result['PREDICTED'],result['ACTUAL'],result['TRAIN_DURATION']

In [ ]:
rmse = np.sqrt(mean_squared_error(result['ACTUAL'], result['PREDICTED']))
# Calculate MAPE %
mape = mean_absolute_percentage_error(result['ACTUAL'], result['PREDICTED']) * 100
# Calculate MPE %
mpe = mean_positive_error(result['ACTUAL'], result['PREDICTED'])
# Calculate MTT in seconds
mtt =duration.sum()

In [ ]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    mtt = np.mean(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i], mtt], index=['	stock_ID','MTT'])
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [ ]:
mape,mpe,metrics_df['MTT'].sum()

(88.36033079421874, 25.706070884662232, 144.97311261057237)

In [ ]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
    # Calculate MAPE %
    mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
    # Calculate MPE %
    mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
    # Calculate MTT in seconds
    mtt = np.sum(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i], mape, mpe, mtt], index=['stock_ID',	'MAPE',	'MPE',	'MTT'])
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [ ]:
# @title Default title text
pd.DataFrame(metrics_df.drop(['stock_ID','MTT'],axis=1).mean()).T

,MAPE,MPE
0,86.242952,24.849202


In [ ]:
metrics_df

,stock_ID,MAPE,MPE,MTT
0,AAPL,24.848700,0.000000,9394.264772
1,AMD,62.172120,41.418167,9394.264772
2,AMZN,16.147588,10.610459,9394.264772
3,BA,23.524982,0.001521,9394.264772
4,BABA,46.090661,36.009108,9394.264772
5,BAC,239.597130,81.329606,9394.264772
6,C,158.548605,72.454607,9394.264772
7,CSCO,164.019546,73.466689,9394.264772
8,CVX,29.078626,0.000000,9394.264772
9,DIS,21.492203,20.359757,9394.264772


In [ ]:
avg_values=metrics_df.mean()

In [ ]:
avg_df = pd.DataFrame(avg_values).T
avg_df